In [6]:
!pip install -U pip setuptools wheel

!pip install -U spacy

!python -m spacy download en_core_web_sm

  Attempting uninstall: wheel
    Found existing installation: wheel 0.37.1
    Uninstalling wheel-0.37.1:
      Successfully uninstalled wheel-0.37.1
  Attempting uninstall: setuptools
    Found existing installation: setuptools 61.2.0
    Uninstalling setuptools-61.2.0:
      Successfully uninstalled setuptools-61.2.0
  Attempting uninstall: pip
    Found existing installation: pip 21.2.4
    Uninstalling pip-21.2.4:
      Successfully uninstalled pip-21.2.4


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
spyder 5.1.5 requires pyqt5<5.13, which is not installed.
spyder 5.1.5 requires pyqtwebengine<5.13, which is not installed.
conda-repo-cli 1.0.4 requires pathlib, which is not installed.
anaconda-project 0.10.2 requires ruamel-yaml, which is not installed.


     ---------------------------------------- 12.8/12.8 MB 2.0 MB/s eta 0:00:00
[+] Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [7]:
!pip install pycrf
!pip install sklearn-crfsuite
!pip install spacy


import spacy
import sklearn_crfsuite
from sklearn_crfsuite import metrics


model = spacy.load("en_core_web_sm")

In [17]:
## Converting words to sentence
def tokens_to_sentence(input_f):
    file_line = input_f.readlines()
    input_f.close()
    final_sentence_list = []

    sentence = ""

    for sent in file_line:
        sent = sent.strip()
        
        if sent == "":
            final_sentence_list.append(sentence) 
            sentence = "" 
        else:
            if sentence:
                sentence += " " + sent
            else:
                sentence = sent
                
    return final_sentence_list

In [33]:
import os
current_directory = os.getcwd()
print(current_directory)



C:\Users\Aniket Kale


In [34]:
## Reading input files
train_sent = open('train_sent','r')
train_sentence=tokens_to_sentence(train_sent)
test_sent=open('test_sent','r')
test_sentence=tokens_to_sentence(test_sent)
train_label=open('train_label','r')
train_label=tokens_to_sentence(train_label)
test_label=open('test_label','r')
test_label=tokens_to_sentence(test_label)

In [35]:
## First 5 sentence of the Train dataset
for i in range(0,5):
    print('Line %d' %(i+1) ,':',train_sentence[i])
    print('Label %d' %(i+1),':',train_label[i])

Line 1 : All live births > or = 23 weeks at the University of Vermont in 1995 ( n = 2395 ) were retrospectively analyzed for delivery route , indication for cesarean , gestational age , parity , and practice group ( to reflect risk status )
Label 1 : O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O
Line 2 : The total cesarean rate was 14.4 % ( 344 of 2395 ) , and the primary rate was 11.4 % ( 244 of 2144 )
Label 2 : O O O O O O O O O O O O O O O O O O O O O O O O O
Line 3 : Abnormal presentation was the most common indication ( 25.6 % , 88 of 344 )
Label 3 : O O O O O O O O O O O O O O O
Line 4 : The `` corrected '' cesarean rate ( maternal-fetal medicine and transported patients excluded ) was 12.4 % ( 273 of 2194 ) , and the `` corrected '' primary rate was 9.6 % ( 190 of 1975 )
Label 4 : O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O
Line 5 : Arrest of dilation was the most common indication in both `` corrected '

In [36]:
## First 5 sentence of the Test dataset
for i in range(0,5):
    print('Line %d' %(i+1) ,':',test_sentence[i])
    print('Label %d' %(i+1),':',test_label[i])

Line 1 : Furthermore , when all deliveries were analyzed , regardless of risk status but limited to gestational age > or = 36 weeks , the rates did not change ( 12.6 % , 280 of 2214 ; primary 9.2 % , 183 of 1994 )
Label 1 : O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O
Line 2 : As the ambient temperature increases , there is an increase in insensible fluid loss and the potential for dehydration
Label 2 : O O O O O O O O O O O O O O O O O O O
Line 3 : The daily high temperature ranged from 71 to 104 degrees F and AFI values ranged from 1.7 to 24.7 cm during the study period
Label 3 : O O O O O O O O O O O O O O O O O O O O O O O O
Line 4 : There was a significant correlation between the 2- , 3- , and 4-day mean temperature and AFI , with the 4-day mean being the most significant ( r = 0.31 , p & # 60 ; 0.001 )
Label 4 : O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O
Line 5 : Fluctuations in ambient temperature are in

Count the number of sentences in the processed train and test dataset

In [37]:
## Total number of sentences in train & test dataset
print("No of lines in train dataset is: ",len(train_sentence))
print("No of lines in test dataset is: ",len(test_sentence))

No of lines in train dataset is:  2599
No of lines in test dataset is:  1056


Count the number of lines of labels in the processed train and test dataset.

In [38]:
## Total number of labels in train & test dataset
print("No of lines in train dataset is: ",len(train_label))
print("No of lines in test dataset is: ",len(test_label))

No of lines in train dataset is:  2599
No of lines in test dataset is:  1056


Concept Identification

In [39]:
#We will first explore what are the various concepts present in the dataset. For this, we will use PoS Tagging.

In [40]:
#Extract those tokens which have NOUN or PROPN as their PoS tag and find their frequency

In [41]:
## importing frequency distribution
from nltk import FreqDist

## Method for extracting if the POS tag is a NOUN or PROPN
def extractNounPropn(train_sentence,test_sentence):
    train_test_POS=[]
    for sentn in (train_sentence,test_sentence):
        for sent in sentn:
            for word in model(sent):
                if(word.pos_=='NOUN' or word.pos_=='PROPN'):
                    train_test_POS.append(word.text)
    return train_test_POS

In [42]:
## Words with noun or proper noun as POS tag
train_test_POS=extractNounPropn(train_sentence,test_sentence)
train_test_POS[:10]

['births',
 'weeks',
 'University',
 'Vermont',
 'delivery',
 'route',
 'indication',
 'cesarean',
 'age',
 'parity']

In [43]:
#Print the top 25 most common tokens with NOUN or PROPN PoS tags

In [44]:
FreqDist(train_test_POS).most_common(25)

[('patients', 492),
 ('treatment', 281),
 ('%', 247),
 ('cancer', 200),
 ('therapy', 175),
 ('study', 154),
 ('disease', 142),
 ('cell', 140),
 ('lung', 116),
 ('group', 94),
 ('chemotherapy', 88),
 ('gene', 87),
 ('effects', 85),
 ('women', 77),
 ('results', 77),
 ('use', 75),
 ('risk', 71),
 ('cases', 71),
 ('surgery', 71),
 ('analysis', 70),
 ('rate', 67),
 ('dose', 66),
 ('response', 66),
 ('survival', 65),
 ('children', 64)]

Defining features for CRF

In [45]:
# Let's define the features to get the feature value for one word.
def featuresForOneWord(sentence,original_sent,pos):
    word=sentence[pos]
    tag_word=model(original_sent)
    features=['word.lower=' + word.lower(),
              'word[-3:]=' + word[-3:],
              'word[-2:]=' + word[-2:],
              'word.isupper=%s' %word.isupper(),
              'word.isdigit=%s' %word.isdigit(),
              'word.startsWithCapital=%s' %word[0].isupper(),
              'tag_word.pos_=%s' %tag_word[pos].pos_]
    if(pos>0):
        prev_word=sentence[pos-1]
        features.extend(['prev_word.isupper=%s' %prev_word.isupper(),
                         'prev_word.lower=' + prev_word.lower(),
                         'prev_word.isdigit=%s' %prev_word.isdigit(),
                         'prev_word.startsWithCapital=%s' %prev_word[0].isupper(),
                         'tag_prevword.pos_=%s'%tag_word[pos-1].pos_])
    else:
        features.append('BEG')
    if(pos==len(sentence)-1):
        features.append('END')

    return features

Getting the features

In [46]:
# Write a code to get features for a sentence.
def featuresForOneSentence(sentence):
    sentence_split=sentence.split()
    return [featuresForOneWord(sentence_split,sentence,pos) for pos in range(len(sentence_split))]

In [56]:
# Write a code to get the labels for a sentence.
def labelsForSentence(labels):
    return labels.split()

In [57]:
#Define the features' values for each sentence as input variable for CRF model in test and the train dataset

X_train=[featuresForOneSentence(sentn) for sentn in train_sentence]
X_test=[featuresForOneSentence(sentn) for sentn in test_sentence]

In [58]:
#Define the labels as the target variable for test and the train dataset

Y_train=[labelsForSentence(labels) for labels in train_label]
Y_test=[labelsForSentence(labels) for labels in test_label]

In [59]:
# Build the CRF model.
try:
    crf=sklearn_crfsuite.CRF(max_iterations=100)
    crf.fit(X_train,Y_train)
except AttributeError:
    pass

In [60]:
#Predict the labels of each of the tokens in each sentence of the test dataset that has been pre processed earlier.

Y_pred=crf.predict(X_test)
Y_pred[30]

['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']

In [61]:
#Calculate the f1 score using the actual labels and the predicted labels of the test dataset.

print("f1 score :",metrics.flat_f1_score(Y_test,Y_pred,average='weighted'))

f1 score : 0.9042102390347845


Identifying Diseases and Treatments using Custom NER

In [62]:
Disease_Treatment_dict=dict()

for lbl in range(len(Y_pred)):
    disease=""
    treatment=""
    label=Y_pred[lbl]
    for j in range(len(label)):
        if label[j]=='D':
            disease=disease+test_sentence[lbl].split()[j]+" "
        if label[j]=='T':
            treatment=treatment+test_sentence[lbl].split()[j]+" "
    disease=disease.lstrip().rstrip()
    treatment=treatment.lstrip().rstrip()
    if disease!='' and treatment!='':
        if disease in Disease_Treatment_dict.keys():
            existing_disease_treatment=list(Disease_Treatment_dict[disease]) 
            existing_disease_treatment.append(treatment)
            Disease_Treatment_dict[disease]=existing_disease_treatment
        else:
            Disease_Treatment_dict[disease]=treatment

In [63]:
Disease_Treatment_dict['hereditary retinoblastoma']

'radiotherapy'